In [2]:
import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv

In [3]:
from langchain.chat_models import ChatCohere

In [4]:
load_dotenv()
llm=ChatCohere(cohere_api_key=os.getenv("COHERE_API_KEY"))

d:\MCQGen\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatCohere` was deprecated in LangChain 0.0.30 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import ChatCohere`.
  warn_deprecated(


In [5]:
from langchain.llms import Cohere
from langchain.chains import LLMChain,SequentialChain
from langchain.prompts import PromptTemplate
import PyPDF2


In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ generator. For  {subject} students, Generate {number} multiple choice questions of {difficulty} difficulty 
Make sure the quetions are not repeated and all the questions must be from the given text. Make sure to format your response in given\
format: {responsejson}. Donot use any other words or anything like 'Here are ....' that might make difficult to extract quiz
"""

In [7]:
prompt=PromptTemplate(input_variables=['text','subject','number','difficulty','responsejson'],template=TEMPLATE)

In [8]:
quiz_chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz",verbose=True)

d:\MCQGen\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
RESPONSE_JSON={
    1:{
        'question':'question',
        'options':{
            'a':'choice1',
            'b':'choice2',
            'c':'choice3',
            'd':'choice4'
        },
        'answer':'correct option'
    },
    2:{
        'question':'question',
        'options':{
            'a':'choice1',
            'b':'choice2',
            'c':'choice3',
            'd':'choice4'
        },
        'answer':'correct option'
    },
    3:{
        'question':'question',
        'options':{
            'a':'choice1',
            'b':'choice2',
            'c':'choice3',
            'd':'choice4'
        },
        'answer':'correct option'
    },
    4:{
        'question':'question',
        'options':{
            'a':'choice1',
            'b':'choice2',
            'c':'choice3',
            'd':'choice4'
        },
        'answer':'correct option'
    },
    5:{
        'question':'question',
        'options':{
            'a':'choice1',
            'b':'choice2',
            'c':'choice3',
            'd':'choice4'
        },
        'answer':'correct option'
    }
}

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. For a given mcq quiz for {subject} students, you need to evaluate the complexity of \
    problem and provide a complexity analysis of the quiz in 50 words. If the quiz is too difficult or too easy compared to congnitive \
    ability of students, you can change the difficulty to match the cognitive ability of students. 
    The Quiz:
    {quiz}
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'],template=TEMPLATE2)

In [12]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)


In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','subject','number','difficulty','responsejson'],output_variables=['quiz','review'],verbose=True)

In [14]:
dict_to_json=json.dumps(RESPONSE_JSON)
dict_to_json

'{"1": {"question": "question", "options": {"a": "choice1", "b": "choice2", "c": "choice3", "d": "choice4"}, "answer": "correct option"}, "2": {"question": "question", "options": {"a": "choice1", "b": "choice2", "c": "choice3", "d": "choice4"}, "answer": "correct option"}, "3": {"question": "question", "options": {"a": "choice1", "b": "choice2", "c": "choice3", "d": "choice4"}, "answer": "correct option"}, "4": {"question": "question", "options": {"a": "choice1", "b": "choice2", "c": "choice3", "d": "choice4"}, "answer": "correct option"}, "5": {"question": "question", "options": {"a": "choice1", "b": "choice2", "c": "choice3", "d": "choice4"}, "answer": "correct option"}}'

In [15]:
filepath="D:/MCQGen/data.txt"
with open(filepath,'r') as file:
    TEXT=file.read()

In [16]:
NUMBER=5
SUBJECT='machine learning'
DIFFICULTY='hard'

In [17]:
response=generate_evaluate_chain(
    {
    'text':TEXT,
    'subject':SUBJECT,
    'number':NUMBER,
    'difficulty':DIFFICULTY,
    'responsejson':dict_to_json,
}
)

d:\MCQGen\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:A transformer is a deep learning architecture developed by Google and based on the multi-head attention mechanism, proposed in a 2017 paper "Attention Is All You Need".[1] Text is converted to numerical representations called tokens, and each token is converted into a vector via looking up from a word embedding table.[1] At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism allowing the signal for key tokens to be amplified and less important tokens to be diminished. The transformer paper, published in 2017, is based on the softmax-based attention mechanism proposed by Bahdanau et. al. in 2014 for machine translation,[2][3] and the Fast Weight Controller, similar to a transformer, proposed in 1992.[4][5][6]

Transformers have the advantage of having no recurrent units, and

In [23]:
import ast
quiz=response['quiz']
quiz=ast.literal_eval(quiz)

In [24]:
my_arr=[]
for key,value in quiz.items():
    mcq=value['question']
    options='|'.join([
        f"{option}:{option_value}" for option,option_value in value['options'].items()
    ])
    answers=value['answer']
    my_arr.append({'question':mcq,'options':options,'answer':answers})

In [25]:
my_arr

[{'question': 'What is the primary advantage of Transformers over recurrent neural architectures like LSTM?',
  'options': 'a:They use a multi-layer perceptron for processing tokens.|b:They employ self-attention mechanisms for context understanding.|c:They do not rely on recurrent units, resulting in reduced training times.|d:They use a convolutional neural network for faster token processing.',
  'answer': 'c'},
 {'question': 'Which of the following best describes the input process for Transformers?',
  'options': 'a:Text is converted to tokens, and each token is embedded into a vector space.|b:Text is processed as a whole, and then divided into token embeddings.|c:Text is first converted to an image, then tokenized and embedded.|d:Text is pre-processed using a recurrent neural network, then tokenized and embedded.',
  'answer': 'a'},
 {'question': 'The Transformer architecture is based on which of these attention mechanisms?',
  'options': 'a:Hard attention|b:Soft attention|c:Softmax

In [26]:
import pandas as pd
quiz_df=pd.DataFrame(my_arr)
quiz_df.to_csv('quiz.csv',index=False)